In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('../')
import experiments

# for plotting across sizes ###################################################
num_net_types = 6
net_type_labels = ['Unregularized', 'All Regularizers', 'Data Augment', 'Dropout', 'Weight Decay', 'Random Labels']
net_type_colors = ['blue', 'green', 'darkviolet', 'maroon', 'orange', 'red']
net_type_styles_styles = ['-', '-', '--', '--', '--', '-']

num_sizes = 5
size_factors = [0.25, 0.5, 1, 2, 4]
size_idx_map = {0.25: 0, 0.5: 1, 1: 2, 2: 3, 4: 4}
size_labels = ['1/4x', '1/2x', '1x', '2x', '4x']

# for plotting across layers ###################################################
num_layerwise_curves = 9
layerwise_labels = ['Unregularized 1/4x', 'Unregularized 1x', 'Unregularized 4x', 'All Regularizers 1/4x',
                'All Regularizers 1x', 'All Regularizers 4x', 'Random Labels 1/4x', 'Random Labels 1x',
                'Random Labels 4x']
layerwise_colors = ['lightblue', 'blue', 'darkblue', 'lightgreen', 'limegreen', 'darkgreen', 'salmon', 'red', 'darkred']
layerwise_styles = ['-', '-', '-', '-', '-', '-', '-', '-', '-']
num_layers = 4  # 4 layers plus and 'all' layer with the averages
layer_labels = [1, 2, 3, 4]
num_crosses = 3

# for plotting robustness ###################################################

num_levels = 7
levels = np.linspace(0.0, 1.0, num=num_levels).tolist()
perturbation_range = np.linspace(0.0, 1.0, num=num_levels)
level_labels = [str(levels[i] for i in range(num_levels))]

# results ###################################################

# idx: [train/test][net_type][size][layer]
accuracy_results = np.zeros((2, num_net_types, num_sizes))

# idx: [train/test][net_type][size][layer][cross][mean/std]
similarity_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses, 2))  
selectivity_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses, 2))

# idx: [train/test][net_type][size][layer][cross]
compressability_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses))
nonselectivity_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses))

# idx: [train/test][net_type][size][layer][perturbation_level][cross]
rand_ablation_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_levels, num_crosses))
noise_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_levels, num_crosses))
targeted_ablation_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_levels, num_crosses))

# get data ###################################################

for ID in range(62, 92):

    print('Processing ID:', ID)
    opt = experiments.opt[ID]

    net_idx = 0  # unregularized
    if opt.dataset.random_labels:
        net_idx = 5  # random labels
    elif opt.hyper.augmentation and opt.hyper.drop_train < 1:
        net_idx = 1  # all regularizers
    elif opt.hyper.augmentation:
        net_idx = 2  # augmentation
    elif opt.hyper.drop_train < 1:
        net_idx = 3  # dropout
    elif opt.hyper.weight_decay:
        net_idx = 4  # weight decay
    size_idx = size_idx_map[opt.dnn.neuron_multiplier[0]]
    
    redundancy_file = experiments.opt[ID].csv_dir + experiments.opt[ID].name + '_redundancy.csv'
    redundancy_df = pd.read_csv(redundancy_file)
    robustness_file = experiments.opt[ID].csv_dir + experiments.opt[ID].name + '_robustness.csv'
    robustness_df = pd.read_csv(robustness_file)
        
    for eval_idx, eval_set in zip(range(2), ['train', 'test']):
        
        accuracy_results[eval_idx, net_idx, size_idx] = \
        redundancy_df[redundancy_df['evaluation_set'] == eval_set]['performance'].iloc[0]
                
        for layer_idx, layer in zip(range(num_layers + 1), list(range(num_layers)) + ['all']):
            
            for cross in range(num_crosses):
                                            
                similarity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 0] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['similarity_ave'].iloc[0]
                similarity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 1] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['similarity_std'].iloc[0]
                
                compressability_results[eval_idx, net_idx, size_idx, layer_idx, cross] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['compressability_95'].iloc[0]
                
                selectivity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 0] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['selectivity_mean'].iloc[0]
                selectivity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 1] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['selectivity_std'].iloc[0]
                
                nonselectivity_results[eval_idx, net_idx, size_idx, layer_idx, cross] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['not_selective'].iloc[0]
                
                for level_idx, level in zip(range(num_levels), levels):
                    
                    rand_ablation_results[eval_idx, net_idx, size_idx, layer_idx, level_idx, cross] = \
                    robustness_df[robustness_df['cross_validation'] == cross] \
                    [robustness_df['evaluation_set'] == eval_set][robustness_df['perturbation_layer'] == str(layer)] \
                    [robustness_df['perturbation_name'] == 'Activation Knockout'] \
                    [robustness_df['perturbation_amount'] == level]['performance'].iloc[0]
                    
                    noise_results[eval_idx, net_idx, size_idx, layer_idx, level_idx, cross] = \
                    robustness_df[robustness_df['cross_validation'] == cross] \
                    [robustness_df['evaluation_set'] == eval_set][robustness_df['perturbation_layer'] == str(layer)] \
                    [robustness_df['perturbation_name'] == 'Activation Noise'] \
                    [robustness_df['perturbation_amount'] == level]['performance'].iloc[0]
                    
                    targeted_ablation_results[eval_idx, net_idx, size_idx, layer_idx, level_idx, cross] = \
                    robustness_df[robustness_df['cross_validation'] == cross] \
                    [robustness_df['evaluation_set'] == eval_set][robustness_df['perturbation_layer'] == str(layer)] \
                    [robustness_df['perturbation_name'] == 'Activation Knockout Selected'] \
                    [robustness_df['perturbation_amount'] == level]['performance'].iloc[0]
                        
accuracy_results *= 100
compressability_results *= 100
nonselectivity_results *= 100
rand_ablation_results *= 100
noise_results *= 100
targeted_ablation_results *= 100

Processing ID: 62
0 0 0 0 0
0 0 0 1 0
0 0 0 2 0
0 0 0 0 1
0 0 0 1 1
0 0 0 2 1
0 0 0 0 2
0 0 0 1 2
0 0 0 2 2
0 0 0 0 3
0 0 0 1 3
0 0 0 2 3
0 0 0 0 4
0 0 0 1 4
0 0 0 2 4
1 0 0 0 0
1 0 0 1 0
1 0 0 2 0
1 0 0 0 1
1 0 0 1 1
1 0 0 2 1
1 0 0 0 2
1 0 0 1 2
1 0 0 2 2
1 0 0 0 3
1 0 0 1 3
1 0 0 2 3
1 0 0 0 4
1 0 0 1 4
1 0 0 2 4
Processing ID: 63
0 5 0 0 0
0 5 0 1 0
0 5 0 2 0
0 5 0 0 1
0 5 0 1 1
0 5 0 2 1
0 5 0 0 2
0 5 0 1 2
0 5 0 2 2
0 5 0 0 3
0 5 0 1 3
0 5 0 2 3
0 5 0 0 4
0 5 0 1 4
0 5 0 2 4
1 5 0 0 0
1 5 0 1 0
1 5 0 2 0
1 5 0 0 1
1 5 0 1 1
1 5 0 2 1
1 5 0 0 2
1 5 0 1 2
1 5 0 2 2
1 5 0 0 3
1 5 0 1 3
1 5 0 2 3
1 5 0 0 4
1 5 0 1 4
1 5 0 2 4
Processing ID: 64
0 2 0 0 0
0 2 0 1 0
0 2 0 2 0
0 2 0 0 1
0 2 0 1 1
0 2 0 2 1
0 2 0 0 2
0 2 0 1 2
0 2 0 2 2
0 2 0 0 3
0 2 0 1 3
0 2 0 2 3
0 2 0 0 4
0 2 0 1 4
0 2 0 2 4
1 2 0 0 0
1 2 0 1 0
1 2 0 2 0
1 2 0 0 1
1 2 0 1 1
1 2 0 2 1
1 2 0 0 2
1 2 0 1 2
1 2 0 2 2
1 2 0 0 3
1 2 0 1 3
1 2 0 2 3
1 2 0 0 4
1 2 0 1 4
1 2 0 2 4
Processing ID: 65
0 3 0 0 0
0 3 0 1 0
0 3 0 2 

/cm/shared/openmind/anaconda/2.5.0/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)
